In [1]:
import numpy as np
import itertools
import math
import random
from NecromundaModel import *
from necromundaStaticValues import *

def getPermutations(number):
    return list(itertools.combinations_with_replacement([i+1 for i in range(6)], number))

def throwAttackDice(roll):
    """
    Should be recieving a tuple of roll values,
    and returning the value of that attack roll,
    fumbles, and criticals.
    returns (maxRoll, fumbles, criticals)
    """
    maxRoll = getMaxRoll(roll)
    criticals = getCriticalHits(roll)
    fumbles = getFumbles(roll)

    return maxRoll, fumbles, criticals

def getMaxRoll(roll):
    return max(roll)

def getCriticalHits(roll):
    criticalHits = -1
    for rollValue in roll:
        if rollValue == 6:
            criticalHits += 1
    if criticalHits < 0:
        criticalHits = 0
    return criticalHits

def getFumbles(roll):
    fumbles = 0
    for rollValue in roll:
        if rollValue == 1:
            fumbles += 1
    return fumbles

def toHitProbability(fighter1, fighter2, charge):
    bonus = 0
    if charge:
        bonus = charge
    p1 = getPermutations(fighter1.a)
    p2 = getPermutations(fighter2.a)
    fights = list(itertools.product(p1,p2))
    
    p1Wins = 0.0
    p2Wins = 0.0
    push = 0.0
    newFights = fights
    p1NumberOfAttacks = 0
    p2NumberOfAttacks = 0

    #print "fighter1 number of parries: {0}".format(fighter1.numberOfParries())
    #print "fighter2 number of chains: {0}".format(fighter2.numberOfChains())

    for i in xrange(0, fighter1.numberOfParries() - fighter2.numberOfChains()):
        newFights = parry(fighter1, fighter2, newFights)
    numFights = len(newFights)
    for fight in newFights:
        maxP1, fumble1, crit1 = throwAttackDice(fight[0])
        maxP2, fumble2, crit2 = throwAttackDice(fight[1])
        if fighter1.numberOfChains() > 0:
            fumble1 *= 2 #+= fighter1.numberOfChains()
        if fighter2.numberOfChains() > 0:
            fumble2 *= 2 #+= fighter2.numberOfChains()
        combatScore1 = maxP1+fighter1.ws+bonus+fumble2+crit1
        combatScore2 = maxP2+fighter2.ws+bonus+fumble1+crit2
        difference = combatScore1 - combatScore2

        if difference > 0:
            p1Wins += 1
            p1NumberOfAttacks += difference#*woundProbability(fighter1, fighter2) <- failure to wound chance to power of attacks
        elif difference < 0:
            p2Wins += 1
            p2NumberOfAttacks += -1*difference#*woundProbability(fighter2, fighter1)
        elif fighter1.i > fighter2.i:
            p1Wins += 1
            p1NumberOfAttacks += 1#woundProbability(fighter1, fighter2)
        elif fighter1.i < fighter2.i:
            p2Wins +=1
            p2NumberOfAttacks += 1#woundProbability(fighter2, fighter1)
        else:
            push +=1

    return numFights, p1NumberOfAttacks, p2NumberOfAttacks, p1Wins/numFights*1.0, p2Wins/numFights*1.0, push/numFights*1.0

def combatScores(attacker, defender, roll, chargeBonus):
    attackerMaxRoll, attackerFumbles, attackerCriticals = throwAttackDice(roll[0])
    defenderMaxRoll, defenderFumbles, defenderCriticals = throwAttackDice(roll[1])
    attackerFumbles = addFumbles(attacker, attackerFumbles)
    defenderFumbles = addFumbles(defender, defenderFumbles)

    attackerCombatScore = attackerMaxRoll+attacker.ws+defenderFumbles+attackerCriticals+chargeBonus
    defenderCombatScore = defenderMaxRoll+defender.ws+attackerFumbles+defenderCriticals

    return attackerCombatScore, defenderCombatScore 

def numberOfAttacks(attacker, defender, attackerCombatScore, defenderCombatScore):
    attacks = 0
    if attackerCombatScore > defenderCombatScore:
        attacks = attackerCombatScore - defenderCombatScore
    elif attackerCombatScore < defenderCombatScore:
        attacks = defenderCombatScore - attackerCombatScore
    elif attacker.i != defender.i:
        attacks = 1

    return attacks

def addFumbles(fighter, fumbles):
    chains = fighter.numberOfChains()
    if chains > 0:
        if fumbles > chains:
            fumbles += chains
        else:
            fumbles *= 2
    return fumbles

def wounds(strength, toughness):
    difference = toughness - strength
    target = 4 + difference
    if target < 2:
        target = 2
    woundRoll = getPermutations(1)
    wounded = 0
    for roll in woundRoll:
        if roll[0] >= target:
            wounded += 1
    return wounded

def woundProbability(strength, toughness):
    return wounds(strength, toughness)/6.0

def armorProbability(save):
    saved = 0.0
    saveRoll = getPermutations(1)
    for roll in saveRoll:
        if roll[0] >= save:
            saved += 1.0
    return saved/6.0

def test(characteristic):
    roll = d6()
    result = roll <= characteristic
    if(characteristic >= 6):
        roll2 = random.randint(1,6)
        result = roll2 >= 3
    return result

def leadershipCheck(fighter):
    roll = d6() + d6()
    return roll <= fighter.l

def d6():
    return random.randint(1,6)

def d3():
    return random.randint(1,6)

def d66():
    return (d6(),d6())

def handleParries(attacker, defender, fights):
    netParries = netNumberOfParries(attacker, defender)
    newFights = []
    if netParries > 0:
        newFights = parry(fights, 0, 1)
    elif netParries < 0:
        newFights = parry(fights, 1, 0)
    else:
        newFights = fights
    return newFights

def parry(fights, parrier, pariee):
    newFights = []
    for fight in fights:
        if getMaxRoll(fight[parrier]) < getMaxRoll(fight[pariee]) and getMaxRoll(fight[parrier]) > 3:
            for i in xrange(0,6):
                parriedFight = list(fight[pariee])
                bestFight = parriedFight.index(getMaxRoll(fight[pariee]))
                parriedFight[bestFight] = i+1
                newFight = (fight[parrier], tuple(parriedFight))
                newFights.append(newFight)
        else:
            newFights.append(fight)

    return newFights

def netNumberOfParries(aggressor, defender):
    parries = aggressor.numberOfParries()
    if isinstance(parries, int):
        parries -= defender.numberOfParries() #sword parries counter each other before nullifications
        parries -= defender.numberOfNullifications() # nullifications come after parries
    return parries

def handToHand(attacker, defender, charge):
    attackerRolls = getPermutations(attacker.a)
    defenderRolls = getPermutations(defender.a)
    fights = list(itertools.product(attackerRolls, defenderRolls))
    fights = handleParries(attacker, defender, fights)

    chargeBonus = 0
    if charge:
        chargeBonus = 1

    attackerWins = 0
    defenderWins = 0
    ties = 0
    numberOfFightPossibilities = len(fights)

    numberOfAttackersAttacks = 0
    attackerAttacks = [0] * 20

    numberOfDefendersAttacks = 0
    defenderAttacks = [0] * 20

    attackerWounds = 0
    defenderWounds = 0
    numberOfWoundPossibilities = 0

    for fight in fights:
        attackerCombatScore, defenderCombatScore = combatScores(attacker, defender, fight, chargeBonus)
        woundAttempts = numberOfAttacks(attacker, defender, attackerCombatScore, defenderCombatScore)
        numberOfWoundPossibilities += woundAttempts*6
        if attackerCombatScore > defenderCombatScore:
            attackerWins += 1
            attackerAttacks[woundAttempts] += 1
            for i in range(woundAttempts):
                attackerWounds += wounds(attacker.getStrength(), defender.t)
                numberOfAttackersAttacks += 6
        elif attackerCombatScore < defenderCombatScore:
            defenderWins += 1
            defenderAttacks[woundAttempts] += 1
            for i in range(woundAttempts):
                defenderWounds += wounds(defender.getStrength(), attacker.t)
                numberOfDefendersAttacks += 6
        elif defender.mainHand.weaponId == 'massiveClub':
            attackerWins += 1
            attackerAttacks[1] += 1
            numberOfAttackersAttacks += 6
            attackerWounds += wounds(attacker.getStrength(), defender.t)
        elif attacker.mainHand.weaponId == 'massiveClub':
            defenderWins += 1
            defenderAttacks[1] += 1
            numberOfDefendersAttacks += 6
            defenderWounds += wounds(defender.getStrength(), attacker.t)
        elif attacker.i > defender.i:
            attackerWins += 1
            attackerAttacks[1] += 1
            numberOfAttackersAttacks += 6
            attackerWounds += wounds(attacker.getStrength(), defender.t)
        elif attacker.i < defender.i:
            defenderWins += 1
            defenderAttacks[1] += 1
            numberOfDefendersAttacks += 6
            defenderWounds += wounds(defender.getStrength(), attacker.t)
        else:
            ties += 1

    wins = (attackerWins, defenderWins, ties)
    attacks = (attackerAttacks, defenderAttacks)
    wound = (attackerWounds, defenderWounds)
    numbers = (numberOfFightPossibilities, numberOfWoundPossibilities, numberOfAttackersAttacks, numberOfDefendersAttacks)
    return wins, attacks, wound, numbers

def shoot(attacker, cover):
    toHitBonus = attacker.mainHand.shortToHit - cover
    rolls = getPermutations(1)
    hits = 0
    for roll in rolls:
        if roll[0] > 1:
            if roll[0]+toHitBonus+attacker.bs >= 7:
                hits += 1
    return hits

def prob(attacks, pWound):
    sumMiss = 0.0

    for i in xrange(1, len(attacks)):
        if(attacks[i] != 0):
            wounding = 1.0 - pWound
            amount = math.pow(wounding, i)
            amountTimes = attacks[i]
            sumMiss += amount * amountTimes
            #print i, amount, amountTimes, sumMiss
    return 1-(sumMiss/sum(attacks))

In [9]:
jimmy = Fighter(pitFighter)
jimmy.name = 'jimmy'
#jimmy.ws = 6
#jimmy.s = 5
#jimmy.t = 5
#jimmy.w = 4
#jimmy.i = 5
jimmy.a = 2
#jimmy.ld = 9
#jimmy.setWeapon(knife)
jimmy.equipWeapon(chainSword, 'mainHand')

bob = Fighter(ganger)
bob.name = 'bob'
bob.ws = 2
#bob.s = 6
#bob.t = 5
#bob.w = 4
#bob.i = 5
#bob.a = 3
#bob.ld = 8
#bob.equipWeapon(sword, 'mainHand')
#bob.equipWeapon(sword, 'offHand')

charge = False

wins, attacks, woundz, numbers = handToHand(jimmy, bob, charge)

winsPercent = (0.0+wins[0])/(0.0+numbers[0])
losePercent = (0.0+wins[1])/(0.0+numbers[0])
woundPercent = prob(attacks[0], woundProbability(jimmy.getStrength(), bob.t))
woundedPercent = prob(attacks[1], woundProbability(bob.getStrength(), jimmy.t))
print "win %: {0}".format(winsPercent)
print "lose %: {0}".format(losePercent)
print "wound %: {0}".format(woundPercent)
print "wounded %: {0}".format(woundedPercent)
print "wound given win %: {0}".format(winsPercent * woundPercent)
print "wounded given lose %: {0}".format(losePercent * woundedPercent)

win %: 0.916230366492
lose %: 0.0837696335079
wound %: 0.90267830833
wounded %: 0.666015625
wound given win %: 0.827061277266
wounded given lose %: 0.0557918848168


In [ ]:
jack = Fighter(ganger)
jack.name = 'jack'
jack.equipWeapon(autopistol, 'mainHand')

print shoot(jack, 2)
print wounds(jack.getStrength(), 3)
print shoot(jack, 2)/6.0 * wounds(jack.getStrength(), 3)/6.0